# Kafi: Kafka Superpowers for Your Jupyter Notebook and Python
<img src="pix/kafka.jpg" style="width: 30%; height: 30%"/>
<img src="pix/jupyter.jpg" style="width: 30%; height: 30%"/>

### Ralph Debusmann
##### `ralph.debusmann@mgb.ch`

<img src="pix/migros.png" style="width: 20%; height: 20%"/>


# Agenda

* Part I: The Birth of Kafi

* Part II: Three Paradigms for Using Kafi
  * Shell/Python interpreter
  * Juypter Notebooks
  * Code (Microservices, FaaS, Agents...)

* Part III: Use Cases for Kafi
  * Kafka Administration
  * Schema Registry Administration
  * Kafka Backups incl. Kafka Emulation
  * Simple Stream Processing
  * Kafka via REST Proxy
  * Building a Bridge from Kafka to Pandas Dataframes and Files

# Part I: The Birth of Kafi


<img src="pix/birth.jpg" style="width: 35%; height: 35%"/>


What happens if you would just like to create a topic on Kafka, list topics, produce some messages, or consume some messages, or search for messages?

The answer is often:
* kafkacat/kcat
* standard Kafka commandline tools (kafka-console-producer, kafka-console-consumer...)


It works...for a long time indeed. But how?

## Still the State-of-the-Art Developer Experience

### List Topics

```
kcat -b localhost:9092 -L
```


```
kafka-topics --bootstrap-server localhost:9092 --list
```

### Create Topics

(not possible with kcat)


```
kafka-topics --bootstrap-server localhost:9092 --topic topic_json1 --create
```


### Produce Messages

```
kcat -b localhost:9092 -t topic_json1 -P -K ,

123,{"bla":123}
456,{"bla":456}
789,{"bla":789}
```


```
kafka-console-producer --bootstrap-server localhost:9092 --topic topic_json1 --property parse.key=true --property key.separator=','

123,{"bla":123}
456,{"bla":456}
789,{"bla":789}
```


### Produce Messages Using a Schema

(not even possible with kcat...)

```
kafka-avro-console-producer --bootstrap-server localhost:9092 --topic topic_avro1 --property schema.registry.url=http://localhost:8081 --property key.serializer=org.apache.kafka.common.serialization.StringSerializer --property value.serializer=io.confluent.kafka.serializers.KafkaAvroSerializer --property value.schema='{"type":"record","name":"myrecord","fields":[{"name":"bla","type":"int"}]}' --property parse.key=true --property key.separator=','

123,{"bla": 123}
456,{"bla": 456}
789,{"bla": 789}
```


### Consume Messages

```
kcat -b localhost:9092 -t topic_json1 -C -o beginning
```


```
kafka-console-consumer --bootstrap-server localhost:9092 --topic topic_json1 --property print.key=true --from-beginning
```

### Search Messages

```
kcat -b localhost:9092 -t topic_json1 -C -o beginning -e | grep 456
```


```
kafka-console-consumer --bootstrap-server localhost:9092 --topic topic_json1 --from-beginning | grep 456
```

## Can't We Do Better?

I developed Kafi because I was frustrated with kcat and the standard Kafka commandline tools. Not by another commandline tool, but by building a Python module (=library) wrapped around Confluent's Python client for Kafka, confluent_kafka.

Regardless of whether you use Kafi in your shell or in a Jupyter notebook, you have a similar experience. And your life gets so much better. I promise.


This is how you can list topics, create topics, produce messages, consume messages or search for messages with Kafi.

Because Kafi is a Python module, you first need to import it. Then, you create a Cluster object `c` reading from a configuration file:

```
from kafi.kafi import *
c = Cluster("local")
```

Then...

### List Topics

```
c.ls()
```

Many commands also support wildcards, so like in a shell, you can do e.g.:
```
c.ls("*off*")
```

### Create Topics

```
c.touch("topic_json2")
```

### Produce Messages

```
pr = c.producer("topic_json2")
pr.produce({"bla": 123}, key="123")
pr.produce({"bla": 456}, key="456")
pr.produce({"bla": 789}, key="789")
pr.close()
```

### Produce Messages Using a Schema

```
t = "topic_avro2"
s = '{"type":"record","name":"myrecord","fields":[{"name":"bla","type":"int"}]}'

pr = c.producer(t, value_type="avro", value_schema=s)
pr.produce({"bla": 123}, key="123")
pr.produce({"bla": 456}, key="456")
pr.produce({"bla": 789}, key="789")
pr.close()
```


### Consume Messages

```
c.cat("topic_json2")
```

or...

```
c.cat("topic_avro2")
```

ok, it's Avro...

```
c.cat("topic_avro2", value_type="avro")
```


### Configuration



Kafi supports the full range of configuration options of Confluent's Python client. This is, for example, the simple configuration file to connect to a local Kafka cluster that we used in our first steps with Kafi before:

```
kafka:
  bootstrap.servers: localhost:9092

schema_registry:
  schema.registry.url: http://localhost:8081
```


...and this is a configuration file for connecting to Confluent Cloud:

```
kafka:
  bootstrap.servers: ${KAFI_KAFKA_SERVER}
  security.protocol: SASL_SSL
  sasl.mechanisms: PLAIN
  sasl.username: ${KAFI_KAFKA_USERNAME}
  sasl.password: ${KAFI_KAFKA_PASSWORD}
  
schema_registry:
  schema.registry.url: ${KAFI_SCHEMA_REGISTRY_URL}
  basic.auth.credentials.source: USER_INFO
  basic.auth.user.info: ${KAFI_SCHEMA_REGISTRY_USER_INFO}
```

There are many other configuration options to fine-tune your cluster connection and to override Kafi's "common sense" defaults (e.g. setting the `auto.offset.reset` to `earliest`). These defaults are one of the building blocks responsible for making it so convenient to use.

# Part II: Three Paradigms for Using Kafi

<img src="pix/paradigms.jpg" style="width: 35%; height: 35%"/>


Wait, this talk is titled "Kafka Superpowers for Your Jupyter Notebook and Python". So where is Kafi in the Jupyter notebook? Ok, here, but that's not what you probably ask yourselves... so far, we just used in the Python interpreter in the shell...

There are actually three main paradigms for using Kafi.

## Shell/Python Interpreter

The first is in your shell using the Python interpreter, like we did in Part I above. That gives you a user/developer experience similar to bash/zsh + kcat or the standard Kafka commandline tools.

## Code (Microservices, FaaS, Agents...)

As Kafi is just a Python module, it is also super useful to use in your Python code. Either for smaller scripts, or even for building microservices, FaaS-functions, or agents (put in a pinch of llamaindex agents for example).

## Jupyter Notebooks

Now finally to them. You will see soon in Part III that Jupyter notebooks are a very convenient and powerful paradigm of using Kafi, especially for Python/Jupyter afficionados like Data Scientists etc.

But... using Kafi in a Jupyter notebook is actually also very convenient and powerful for Kafka administrators or developers! You'll see.

# Part III: Use Cases for Kafi


<img src="pix/use_cases.jpg" style="width: 35%; height: 35%"/>


## Kafka Administration

We covered a bit of that already when we compared Kafi to kcat/the standard Kafka commandline tools. So let's start again by importing Kafi and connecting to our local Kafka cluster, and a Confluent Cloud cluster for good measure.

In [ ]:
from kafi.kafi import *
cl = Cluster("local")
cc = Cluster("ccloud")

### Brokers

A basic administration task is to show the brokers of your Kafka cluster. So let's view the brokers of our local Kafka cluster first.

In [ ]:
cl.brokers()

Interesting, now for our Confluent Cloud Basic Cluster...

In [ ]:
cc.brokers()

How about the broker configs?

In [ ]:
cl.broker_config()

For our Confluent Cloud cluster, we only want to see the config of one broker, broker 11:

In [ ]:
cc.broker_config(11)

What if we'd just like to see one configuration item, e.g. `message.max.bytes`?

In [ ]:
_[11]["message.max.bytes"]

We can just as well change this configuration item, or at least try to do it.

In [ ]:
cc.broker_config(config={"message.max.bytes": 1048582})

Well there we are. But it should work on our local cluster:

In [ ]:
cl.broker_config(config={"message.max.size": 1048582})

### Consumer Groups

Let's first see the consumer groups that we have.

In [ ]:
cl.gls()

That's the automatically created groups that our `cat` command created before. What are the offsets of one of them?

In [ ]:
g = "..."

cl.group_offsets(g)


What if we need the same consumer group offsets for another consumer group... on Confluent Cloud? For that, let's first create a topic on Confluent Cloud and populate it.

In [ ]:
t = "topic_json2"

cc.touch(t)
pr = cc.producer(t)
pr.produce({"bla": 123}, key="123")
pr.produce({"bla": 456}, key="456")
pr.produce({"bla": 789}, key="789")
pr.close()

Go. Copy the offsets of our consumer group on our local cluster to a new one on Confluent Cloud.

In [ ]:
cl.cp_group_offsets(t, g, cc, g)

Check it...

In [ ]:
cc.group_offsets(g)

Let's close by going back to our local cluster and deleting the source consumer groups.

In [ ]:
cl.grm("17*")

In [ ]:
cl.gls()

### Topics

As for the brokers, we can have a look at the configuration of a topic...

In [ ]:
t = "topic_json2"

cl.config(t)

...we can change the configuation just as well:

In [ ]:
cl.config(t, {"retention.ms": -1})

We can create or delete topics...

In [ ]:
cl.touch("abc")


In [ ]:
cl.rm("a*")

We can list topics with their total sizes...

In [ ]:
cl.l()

...see their partitions:

In [ ]:
cl.partitions(t)

...and their watermarks:

In [ ]:
cl.watermarks(t)

Now let's create a new test topic and write some messages to it.

In [ ]:
import time

t = "topic_offsets"

pr = cl.producer(t)
pr.produce({"bla": 123}, key="123")
time.sleep(0.1)
pr.produce({"bla": 456}, key="456")
time.sleep(0.1)
pr.produce({"bla": 789}, key="789")
pr.close()

cl.cat(t)

Let's pick the timestamp of the second message and search for it:

In [ ]:
cl.offsets_for_times(t, {0: ...})

Perfect. Of course, we can also delete some records from the beginning of the topic. E.g. the first two:

In [ ]:
print(cl.watermarks(t))

cl.delete_records({t: {0: 2}})

cl.watermarks(t)


Now we are left with only the third message:

In [ ]:
cl.cat(t)

We can also repeat the last message, e.g. useful for testing consumers without having to reset their consumer group offsets:

In [ ]:
cl.repeat(t, 1)

In [ ]:
cl.cat(t)

...or recreate a topic with the exact same configuration:

In [ ]:
cl.recreate(t)

In [ ]:
cl.l(t)

How about some statistics about one of our still populated topics?

In [ ]:
t = "topic_json2"

cl.message_size_stats(t)

Stay with me... we have some more functionality that goes beyond just having a "cat" command... all shell-inspired...

See n messages from the beginning of the topic:

In [ ]:
cl.head(t, 1)

Or from the end:

In [ ]:
cl.tail(t, 1)

Now it increasingly gets wilder. How about copying a topic from our local cluster to Confluent Cloud?

In [ ]:
t2 = t + "_from_local_cluster"

cc.touch(t2)

cl.cp(t, cc, t2)

cc.l(t2)

Let's see this on Confluent Cloud...

And now, some more shelly stuff.

Word count:

In [ ]:
cl.wc(t)

Do a diff of the topic on our local cluster with that on Confluent Cloud (should be the same):

In [ ]:
cl.diff(t, cc, t)

Do a grep on the topic to find the message with value `456`:

In [ ]:
cl.grep(t, ".*456.*")

...and do a grep with our own lambda function instead of a regular expression:

In [ ]:
cl.grep_fun(t, lambda x: x["key"] == "789")

## Schema Registry Administration

With Kafi, you also have the entire array of the Schema Registry API at your disposal.

List subjects:

In [ ]:
cl.sls()

Or maybe just those matching a pattern (well, in this case we have just one, but it's nice nonetheless).

In [ ]:
cl.sls("*-value")

Get all the versions of the subject:

In [ ]:
s = "topic_avro2-value"

cl.get_versions(s)

Get the latest version...

In [ ]:
cl.get_latest_version(s)

Next. We list the subjects, delete our schema, and list the subjects again:

In [ ]:
print(cl.sls())

cl.srm(s)

cl.sls()

Now let's see if it is only soft-deleted...

In [ ]:
cl.sls(deleted=True)

Aha, so let's kill it off completely.

In [ ]:
cl.srm(s, permanent=True)

Now it should really be gone.

In [ ]:
cl.sls(deleted=True)

The rest of the Schema Registry API is also supported:

* get_schema
* register_schema
* lookup_schema
* get_schema_versions
* get_versions
* delete_version
* get_compatibility
* set_compatibility
* test_compatibility

That's it for the first use case - for doing Kafka administration with Kafi :-)

## Simple Stream Processing

Kafi also offers some functionality for simple stream processing. It's nowhere as expressive and powerful as e.g. Kafka Streams or Flink, or other Python libraries like Quix, Bytewax, Pathway etc. - but for many day-to-day tasks and microservices, this could even be enough.

Oh, and shameless plug. If you wish to read up on stream processing and streaming databases, and the ongoing convergence of streaming and databases/data warehouses/data lakes (e.g. Tableflow) - there is a book that I can recommend ;-)

<img src="pix/sdb.jpg" style="width: 30%; height: 30%"/>


Back to the topic. Kafi and stream processing. All the functionality for stream processing (and actually, even simpler commands like `cat` or `head`) are based on a functional backbone. As a functional programmer, or a Kafka Streams DSL or Flink DataStream API user, you'll feel at home immediately.


We start with `foreach`. Here, we simply read the topic message-by-message and print out its key. We could do anything.

In [ ]:
t = "topic_json2"

pr = cl.producer(t)
pr.produce({"bla": 123}, key="123")
pr.produce({"bla": 456}, key="456")
pr.produce({"bla": 789}, key="789")
pr.close()

In [ ]:
t = "topic_json2"

cl.foreach(t, lambda x: print(x["key"]))

Next, we go a bit further and use a `map` function that reads individual messages from a topic, does a "single message transform", and returns the result of the transformation.

In [ ]:
def add(x):
    x["value"]["bla"] += 1000
    return x

cl.map(t, add)


Of course we can also write out the result of the transformation to another topic, even on another cluster. So let's do the same transformation as above and write the result out to our Confluent Cloud cluster...

In [ ]:
t2 = "topic_json_map"

cc.touch(t2)

cl.map_to(t, cc, t2, add)


Let's see if that has worked...

Next command: `flatmap`. Take individual messages and return a list of them (possibly empty of course). In the following example, we just duplicate the messages.

In [ ]:
def dup(x):
    return [x, x]

cl.flatmap(t, dup)

Again, let's write out the result to Confluent Cloud...

In [ ]:
t3 = "topic_flatmap"

cc.touch(t3)

cl.flatmap_to(t, cc, t3, dup)

...and check out the result there.

The next command, `filter`, is just a special case of `flatmap`. Here, we just want to keep the message with value `456`:

In [ ]:
cl.filter(t, lambda x: x["value"]["bla"] == 456)

Of course, `filter_to` is also there:

In [ ]:
t4 = "topic_filter"

cc.touch(t4)

cl.filter_to(t, cc, t4, lambda x: x["value"]["bla"] == 456)

One more check on Confluent Cloud...

...and continue. `foldl` stands for "fold left" in functional programming, and is often also called `reduce` (e.g. in Kafka Streams). It is useful for simple stateful stream processing.

In the example below, we do a very simple aggregation: We sum up the values.

In [ ]:
def sum(acc, x):
    acc += x["value"]["bla"]
    return acc

cl.foldl(t, sum, 0)

And again, Kafi allows you to write out the result of your processing into another topic. On any cluster. It's a bit more involved though. What we do below is to get the value of each message in the source topic on our local cluster, remove the `bla` field, and add another field `sum` with the current sum:

In [ ]:
t5 = "topic_foldl"

cc.touch(t5)

def sum_to(acc, x):
    acc += x["value"]["bla"]
    #
    y = dict(x)
    del y["value"]["bla"]
    y["value"]["sum"] = acc
    #
    return acc, [y]

cl.foldl_to(t, cc, t5, sum_to, 0)

...and now for the last simple stream processing function.

We join the source topic with the `bla` field from our local Kafka cluster with the new topic with only the `sum` field on Confluent Cloud, and write out the result to our local Kafka cluster to have a topic that has both fields.

We use the key of the messages to join them, as e.g. in Kafka Streams (of course, you can override this and also e.g. use a field in the value payload).

BTW the join code is inspired by DBSP/Feldera, if you don't know it, have a look at e.g. this super cool blog on their web page:
https://www.feldera.com/blog/gpu-stream-dbsp


In [ ]:
t6 = "topic_join"

cl.join_to(t, cc, t5, cl, t6)

Let's check the result:

In [ ]:
cl.cat(t6)

## Kafka via REST Proxy

The entire functionality of Kafi cannot only be used via the direct Kafka protocol, but also via a REST Proxy. This might sometimes be necessary if you have a firewall blocking the Kafka port, or a Private Cluster that you can only access via IP whitelisting.

How does it work? You just create a `RestProxy` object instead of `Cluster`, and then e.g. do a `ls`:

In [ ]:
rl = RestProxy("local")
rl.ls()

Now is this really going over HTTP? Have a look...

In [ ]:
rl.verbose(2)
rl.ls()

Really all the commands that you have seen above also work via REST now. E.g. you can produce to a topic as before...

In [ ]:
t = "topic_rest_proxy"

pr = rl.producer(t)
pr.produce({"bla": 123}, key="123")
pr.produce({"bla": 456}, key="456")
pr.produce({"bla": 789}, key="789")
pr.close()

And do a `cat`:

In [ ]:
rl.cat(t, key_type="json")

Or do some wild thing like reading a topic via REST and copying it over to Confluent Cloud via the direct Kafka protocol:

In [ ]:
trp = "topic_from_rest_proxy"

cc.touch(trp)

rl.cp(t, cc, trp)

Let's go back to that funky UI one more time...


## Kafka Backups incl. Kafka Emulation

Kafi has built-in "Kafka Emulation". That is e.g. extremely useful e.g. for backing up topics to local disk, and replaying them back 1:1 back to Kafka.

Here, we create a `Local` object that points to our local hard disk, and backup a topic from Confluent Cloud to it.


In [ ]:
cc = Cluster("ccloud") 
l = Local("local")

t = "topic_json2"

cc.cp(t, l, t)

Let's check out whether the topic has landed on our "Kafka Emulation":

In [ ]:
l.l()

Cool. Let's see it:

In [ ]:
l.cat(t)

And now, since we have read it, there should be an "emulated" consumer group as well, no?

In [ ]:
l.gls()

In [ ]:
l.describe_groups()


How does this look like under the covers? Let's see.

But Kafi doesn't only support local disk here. You can just as well use Kafi's direct Azure Blob Storage support, or, as we will show, S3.

In [ ]:
s = S3("local")
s.ls()


Still empty. So let's copy the topic from somewhere (e.g. our local Kafka cluster) to S3 (local MinIO).

In [ ]:
cl.cp(t, s, t)

...and?

In [ ]:
s.l()

One more `cat`, now reading from S3:

In [ ]:
s.cat(t)

Let's check this out in the MinIO UI as well...

...and lastly, let's copy back the topic from local S3 "Kafka Emulation" to Confluent Cloud.

In [ ]:
t7 = "topic_json_from_s3"

cc.touch(t7)

s.cp(t, cc, t7)

## Building a Bridge from Kafka to Pandas Dataframes and Files

We are at the end, oh no, one more thing.

Kafi's name doesn't only mean "coffee" in Swiss German, but it actually means "*Ka*fka" and "*Fi*les".

In this sense, you can not only use Kafi for doing backups and play them back to Kafka.


What Kafi can do, is e.g. copy a topic into a Pandas dataframe.

In [ ]:
df = cl.topic_to_df(t)
df

And back to Kafka...

In [ ]:
tdf = "topic_df"

cl.df_to_topic(df, tdf)

Let's check that out... the keys should be missing as these commands yet only use the value of the messages:

In [ ]:
cl.cat(tdf)

Once you have `topic_to_df` and `df_to_topic`, it is not a far step to use all kinds of file formats supported by Pandas:
* csv
* feather
* json
* orc
* parquet
* xslx
* xml

Hence Kafi also supports direct dumping of a topic to a Parquet file for instance, in this example, from the local Kafka cluster to S3:

In [ ]:
s = S3("local")

cl.topic_to_file(t, s, "topic.parquet")


Let's check this out in the MinIO UI...

We can also copy the topic from e.g. from our local Kafka to an Excel file:

In [ ]:
cl.topic_to_file(t, s, "topic.xlsx")

Let's download and have a look...

And, cool thing is, we can go the other way round, too. We can bring back the Excel file on S3 and write it out to a topic on Confluent Cloud...

In [ ]:
texcel = "topic_from_excel"

cc.touch(texcel)

s.file_to_topic("topic.xlsx", cc, texcel)

A final look into the Confluent Cloud UI hopefully shows us that it has worked...

Just head over to GitHub for the Kafi project and its documentation:
https://github.com/xdgrulez/kafi

This Jupyter notebook can also be found there, in case you'd like to go through it yourself:
https://github.com/xdgrulez/cur25blr

<img src="pix/thank_you.jpg" style="width: 60%; height: 60%"/>